In [ ]:
from time import sleep
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

totalurl=[("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=5&page={}&order=S11&view_size=20",7),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=17&page={}&order=S11&view_size=20",7),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=42&page={}&order=S11&view_size=20",6),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=21&page={}&order=S11&view_size=20",4),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=22&page={}&order=S11&view_size=20",4),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=30&page={}&order=S11&view_size=20",3),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=20&page={}&order=S11&view_size=20",3),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=76&page={}&order=S11&view_size=20",10),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=59&page={}&order=S11&view_size=20",5),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=45&page={}&order=S11&view_size=20",4),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=50&page={}&order=S11&view_size=20",3),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=55&page={}&order=S11&view_size=20",3),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=69&page={}&order=S11&view_size=20",3),
("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=44&page={}&order=S11&view_size=20",2)]

df_cars=[]

urls=[]
for i in totalurl:
    pagenum=i[1]
    for j in range(pagenum):
        url=i[0].format(str(j))
        urls.append(url)
        
info=["모델명","연식","주행거리","연료","배기량","변속기","연비","최고출력","최대토크","차량중량","구동방식","보험처리","중고가격"]

for url in urls:
    res=requests.get(url,verify = False,timeout=50000)
    res.raise_for_status()
    requests.adapters.DEFAULT_RETRIES = 10000
    soup=BeautifulSoup(res.text,"lxml")

    cars=soup.find_all("li",attrs={"class":"product-item"})
    links=[]

    for car in cars:
        link = "https://www.bobaedream.co.kr" + car.a["href"]
        links.append(link)
    for link in links:
        print(link)

        res2=requests.get(link,verify=False,timeout=50000)
        res2.raise_for_status()
        soup2 = BeautifulSoup(res2.text, "lxml")
        
        try:
            insurance = soup2.find("b",attrs={"class":"cr"})
            
            state=soup2.find("div",attrs={"class":"tbl-01 st-low"})
            state2=soup2.find("div",attrs={"class":"wrap-detail-spec mode-n6"})
            state3=soup2.find("div",attrs={"class":"top-state"})
            
            name=soup2.find("h3",attrs={"class":"tit"})
            year=state.find("th",text='연식').find_next_sibling("td")
            km=state.find("th",text='주행거리').find_next_sibling("td")
            fuel=state.find("th",text='연료').find_next_sibling("td")
            amount=state.find("th",text='배기량').find_next_sibling("td")
            auto = state.find("th",text='변속기').find_next_sibling("td")
            efficiency = state2.find("span", text="연비").find_next_sibling("strong")
            horsepower = state2.find("span",text='최고출력').find_next_sibling("strong")
            torque = state2.find("span",text='최대토크').find_next_sibling("strong")
            weight = state2.find("span",text='차량중량').find_next_sibling("strong")
            Drive_sys = state2.find("span",text='구동방식').find_next_sibling("strong")
            insurance_num = state3.find("b",attrs={"class":"cr"})
            insurance_num = int(insurance_num.text) if insurance_num is not None else 0
            price=soup2.find("span",attrs={"class":"price"})
            
            temp=[name.get_text(),year.get_text(),km.get_text(),fuel.get_text(),amount.get_text(),
                  auto.get_text(),efficiency.get_text(),horsepower.get_text(),torque.get_text(),
                  weight.get_text(),Drive_sys.get_text(),insurance_num, price.get_text()]
            
            df_cars.append(temp)
            
        except AttributeError:
            continue
        
df_cars=pd.DataFrame(data=df_cars,columns=info)
        
df_cars.to_csv('C:/Users/PC/Documents/data.csv')